# This notebook will teach you how to login to an existing node.

#### We will show you how to
 - get an existing slice
 - get a node from the slice
 - get the IP of the node
 - make the ssh command that you can use to access the node
 
 <hr/>

### Before we start the tutorial, we will first create a slice with one node.

In order to access our VM, we need a key-pair. Your JupyterLab instance has a key-pair in the `/home/fabric/.ssh/` folder.

The `/home/fabric/.ssh/id_rsa.pub` file is the public key, and the `/home/fabric/.ssh/id_rsa` file is the private key.

In [1]:
%ls /home/fabric/.ssh/

id_rsa  id_rsa.pub


We are going to set up the node that create with the public key of the key pair. And then we can later use the private key to access the node.

Let's store the path to the public key in a variable called `public_key_path`, and the private key in a variable called `private_key_path`.

In [2]:
public_key_path = "/home/fabric/.ssh/id_rsa.pub"
private_key_path = "/home/fabric/.ssh/id_rsa"

### Now we create a slice with one node. Note that this is not the focus of this tutorial.

In [3]:
import os
from fabrictestbed.slice_manager import SliceManager, Status
import json

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")
print(os.environ['FABRIC_CREDMGR_HOST'])

CM Host: beta-2.fabric-testbed.net Orchestrator Host: beta-7.fabric-testbed.net
beta-2.fabric-testbed.net


In [4]:
not_found=False
fabric_refresh_token=None
%store -r fabric_refresh_token

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token
print("Fabric Refresh Token {}".format(fabric_refresh_token))
print("CILOGON_REFRESH_TOKEN environment variable: {}".format(os.environ['CILOGON_REFRESH_TOKEN']))

Fabric Refresh Token NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTIYZTGJRDKZRUGNRTOOBTGJRTANZSGU4WEMTFGE2GCZRRMMYDKMJ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRVGA3TIMBUHAYTMOBGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ
CILOGON_REFRESH_TOKEN environment variable: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTCYRZGNRTEMBXG4YDENTEGFRDKMTFGE2DOOBWGAYWENLGGIYDMZJ7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRVGA2TENRTGM3TQMRGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ


In [5]:
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, 
                             refresh_token=fabric_refresh_token, project_name='all', scope='all')

In [6]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("Stored new Refresh Token")
%store fabric_refresh_token
print()
print()


New Refresh Token: NB2HI4DTHIXS6Y3JNRXWO33OFZXXEZZPN5QXK5DIGIXTCOBSHBSTEOBYGY2DKYTDHE4WKZJZME4GGMZSGIYTKNJSMI4GENR7OR4XAZJ5OJSWM4TFONUFI33LMVXCM5DTHUYTMMRVGA3TQNJTGE4DMMRGOZSXE43JN5XD25RSFYYCM3DJMZSXI2LNMU6TQNRUGAYDAMBQ

Stored new Refresh Token
Stored 'fabric_refresh_token' (str)




In [7]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
print(f"Toplogy: {advertised_topology}")

Status: Status.OK
Toplogy: UKY: { cpu: 6/6 , core: 96/96 , ram: 1,536/1,536 G, disk: 109,600/109,600 G, unit: 3/3 }
	Components:
		NVME-P4510:  NVME P4510 { disk: 10,000/10,000 G, unit: 10/10 }
		SharedNIC-ConnectX-6:  SharedNIC ConnectX-6 { unit: 12/12 }
		GPU-RTX6000:  GPU RTX6000 { unit: 2/2 }
		GPU-Tesla T4:  GPU Tesla T4 { unit: 4/4 }
		SmartNIC-ConnectX-6:  SmartNIC ConnectX-6 { unit: 2/2 }
		SmartNIC-ConnectX-5:  SmartNIC ConnectX-5 { unit: 2/2 }
	Site Interfaces:
Links:


In [13]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType

t = ExperimentTopology()

cap = Capacities()
cap.set_fields(core=2, ram=4, disk=100)
# Add node
n1 = t.add_node(name='n2', site='UKY')
# Set properties
n1.set_properties(capacities=cap, image_type='qcow2', image_ref='default_ubuntu_20')

# Generate Slice Graph
slice_graph = t.serialize()

ssh_key = None
with open (public_key_path, "r") as myfile:
    ssh_key=myfile.read()
    ssh_key=ssh_key.strip()

# Request slice from Orchestrator
status, reservations = slice_manager.create(slice_name='JupyterSlice1', slice_graph=slice_graph, ssh_key=ssh_key)

print("Response Status {}".format(status))
print("Reservations created {}".format(reservations))

Response Status Status.OK
Reservations created [{
    "graph_node_id": "2929f561-d518-4baa-9e14-2aac36c1d206",
    "join_state": "None_",
    "lease_end": "2021-07-01 18:44:10",
    "reservation_id": "65e0a5e7-d791-45d7-9da9-8ffc091a61ba",
    "reservation_state": "Unknown",
    "resource_type": "VM",
    "slice_id": "04ea6245-fc22-435a-889f-e5afd4b66f62"
}]


# Tutorial starts here.

### The first thing that we need to do is to get our existing slices.

In [15]:
status, slices = slice_manager.slices()

print("Response Status {}".format(status))
print("Slices {}".format(slices))


Response Status Status.OK
Slices [{
    "graph_id": "ccb4cfd5-8ed1-422d-89ca-c9a9c8b9f86b",
    "slice_id": "04ea6245-fc22-435a-889f-e5afd4b66f62",
    "slice_name": "JupyterSlice1",
    "slice_state": "StableOK"
}]


### We can see that we only have one slice. Let's query the contents of this slice. We need the slice ID of this slice to do that.

In [16]:
slice_id = slices[0].slice_id

### We now have the slice ID in the `slice_id` variable. Now let's query the slivers (contents) of the slice.

In [17]:
status, slivers = slice_manager.slivers(slice_id=slice_id)

print("Response Status {}".format(status))
if status == Status.OK:
    print("Slivers {}".format(slivers))

Response Status Status.OK
Slivers [{
    "allocated_capacities": "{\"core\": 2, \"disk\": 100, \"ram\": 8}",
    "allocated_labels": "{\"instance\": \"instance-00000157\", \"instance_parent\": \"uky-w1\"}",
    "capacities": "{\"core\": 2, \"disk\": 100, \"ram\": 4}",
    "capacity_hints": "{\"instance_type\": \"fabric.c2.m8.d100\"}",
    "graph_node_id": "2929f561-d518-4baa-9e14-2aac36c1d206",
    "join_state": "NoJoin",
    "lease_end": "2021-07-01 18:44:10",
    "management_ip": "128.163.179.58",
    "name": "n2",
    "reservation_id": "65e0a5e7-d791-45d7-9da9-8ffc091a61ba",
    "reservation_state": "Active",
    "resource_type": "VM",
    "site": "UKY",
    "slice_id": "04ea6245-fc22-435a-889f-e5afd4b66f62"
}]


### We can see that the slice contains one thing, which is our node.

### The IP that we can use to access our node is in the `management_ip` variable.

In [18]:
ip_address = slivers[0].management_ip

### Now the IP is in the `ip_address` variable.

### So now, we can print the SSH command that we can use to access the machine.

### We need to use
 - the IP address
 - the path to the private key, which is in the `private_key_path` variable.

In [20]:
print("ssh -i " + private_key_path + " ubuntu@" + ip_address)

ssh -i /home/fabric/.ssh/id_rsa ubuntu@128.163.179.58


### You can use the command above in a terminal to access the VM.